In [1]:
import numpy as np

from pydrake.geometry import StartMeshcat, MeshcatVisualizer
from pydrake.math import RigidTransform, RollPitchYaw, sin

from pydrake.systems.framework import DiagramBuilder, LeafSystem, BasicVector
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph, MultibodyPlant
from pydrake.multibody.parsing import Parser
from pydrake.all import Simulator

# pydot = interface to graphViz,consist in a parser for the DotFormat used by GraphViz
from pydot import graph_from_dot_data

In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [10]:
class Controller(LeafSystem):

    def __init__(self): 
        LeafSystem.__init__(self)
        #self.DeclareVectorInputPort("u", BasicVector(2)) 
        self.DeclareVectorOutputPort("y", BasicVector(2), self.CalcOutputY)
        self.t = 0

    def CalcOutputY(self, context, output): 
        q1dotdot = np.sin(self.t)
        q2dotdot = np.sin(self.t)
        self.t += 0.1
        output.SetFromVector([q1dotdot, q2dotdot])

In [11]:
builder = DiagramBuilder()

time_step = 0.003
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step)
controller = builder.AddNamedSystem("controller",Controller())
Parser(plant, scene_graph).AddModels("../urdf_model/RR_planar.sdf")
plant.Finalize()

builder.Connect(controller.get_output_port(), plant.get_actuation_input_port())



In [12]:
visualizer = MeshcatVisualizer.AddToBuilder(
    builder, 
    scene_graph.get_query_output_port(),
    meshcat
)

diagram = builder.Build()
diagram_root_context = diagram.CreateDefaultContext()
    

In [13]:
plant_context = plant.GetMyContextFromRoot(diagram_root_context)

plant.GetJointByName('q1').set_angle(plant_context, np.pi)
plant.GetJointByName('q2').set_angle(plant_context, np.pi/3)

#plant.get_actuation_input_port().FixValue(plant_context, np.ones(2))
print(plant_context)
diagram.ForcedPublish(diagram_root_context)


::_::plant Context
-------------------
Time: 0
States:
  1 discrete state groups with
     4 states
       3.141592653589793 1.047197551196598                 0                 0

Parameters:
  11 numeric parameter groups with
     1 parameters
       0
     1 parameters
       1
     1 parameters
       0
     1 parameters
       1
     10 parameters
       nan nan nan nan nan nan nan nan nan nan
     10 parameters
       1 0 0 0 1 1 1 0 0 0
     10 parameters
       1 0 0 0 1 1 1 0 0 0
     12 parameters
       1 0 0 0 1 0 0 0 1 0 0 0
     12 parameters
       1 0 0 0 1 0 0 0 1 0 0 0
     12 parameters
         1   0   0   0   1   0   0   0   1 0.5   0   0
     12 parameters
       1 0 0 0 1 0 0 0 1 0 0 0
  3 abstract parameters



In [14]:
simulator = Simulator(diagram, diagram_root_context)
simulator.AdvanceTo(40)
print(plant_context)

::_::plant Context
-------------------
Time: 40
States:
  1 discrete state groups with
     4 states
          2.377642922345155    3.009325374993742 -0.01910157572965246  0.04147465230916341

Parameters:
  11 numeric parameter groups with
     1 parameters
       0
     1 parameters
       1
     1 parameters
       0
     1 parameters
       1
     10 parameters
       nan nan nan nan nan nan nan nan nan nan
     10 parameters
       1 0 0 0 1 1 1 0 0 0
     10 parameters
       1 0 0 0 1 1 1 0 0 0
     12 parameters
       1 0 0 0 1 0 0 0 1 0 0 0
     12 parameters
       1 0 0 0 1 0 0 0 1 0 0 0
     12 parameters
         1   0   0   0   1   0   0   0   1 0.5   0   0
     12 parameters
       1 0 0 0 1 0 0 0 1 0 0 0
  3 abstract parameters



In [8]:
print(plant_context)

::_::plant Context
-------------------
Time: 40
States:
  1 discrete state groups with
     4 states
          2.377642922345155    3.009325374993742 -0.01910157572965246  0.04147465230916341

Parameters:
  11 numeric parameter groups with
     1 parameters
       0
     1 parameters
       1
     1 parameters
       0
     1 parameters
       1
     10 parameters
       nan nan nan nan nan nan nan nan nan nan
     10 parameters
       1 0 0 0 1 1 1 0 0 0
     10 parameters
       1 0 0 0 1 1 1 0 0 0
     12 parameters
       1 0 0 0 1 0 0 0 1 0 0 0
     12 parameters
       1 0 0 0 1 0 0 0 1 0 0 0
     12 parameters
         1   0   0   0   1   0   0   0   1 0.5   0   0
     12 parameters
       1 0 0 0 1 0 0 0 1 0 0 0
  3 abstract parameters



In [15]:
def save_plant_svg_graph(diagram, destination_path):
    dot_format_graph = graph_from_dot_data(diagram.GetGraphvizString())[0]
    binary_data_svg = dot_format_graph.create_svg()
    with open(destination_path, 'wb') as f:
        f.write(binary_data_svg)

save_plant_svg_graph(diagram, destination_path="../log/RR_planar.svg")